<i>Tamer Abousoud</i>

<br><center><span style='font-family: "DejaVu Sans"; font-size:48px; color:#FF7133'>Growing Pains Case</span></center>
<center><span style='font-family: "DejaVu Sans"; font-size:28px'><i>EDA and Data Preparation</i></span></center>

---

In [2]:
# Standard libraries
from __future__ import print_function
import os
from itertools import product, permutations, combinations
from collections import defaultdict
# 3rd party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipyleaflet import Map, Marker, MarkerCluster, AwesomeIcon

---
<br>

<span style='color:red'><b><i>NOTE:</i></b></span> The problem states that Northeast Home Goods (NHG) has <b>123</b> stores but the spreadsheets provided show <b><i>122</i></b> locations plus the Distribution Center (DC) operated by Massachusetts Area Distribution (MAD). The 123<sup>rd</sup> location is assumed to be the DC itself.

---

Make sure the .csv files are all located in the current directory (not necessary but helpful):

In [3]:
data_dir = './data/'
csv_files = [i for i in os.listdir(data_dir) if i.endswith('.csv')]; csv_files

['distances.csv', 'orders.csv', 'locations.csv']

<br> Create dataframes of the store locations, orders and distances:

In [8]:
# Store locations
store_locs = pd.read_csv('data/locations.csv', nrows=123)  # last few rows in .csv are junk
# Change zipcodes to strings and pad with leading zeros
store_locs['ZIP'] = store_locs['ZIP'].apply(lambda x: str(int(x)).zfill(5)) 
store_locs.head(10)

,ZIP,X,Y,CITY,STATE,ZIPID
0,01060,-72.631389,42.318611,Northampton,MA,1
1,01101,-72.578056,42.106111,Springfield,MA,2
2,01420,-71.802222,42.583611,Fitchburg,MA,3
3,01510,-71.682778,42.416667,Clinton,MA,4
4,01570,-71.885556,42.048056,Webster,MA,5
5,01581,-71.615833,42.269444,Westborough,MA,6
6,01606,-71.819167,42.292500,Worcester,MA,7
7,01701,-71.415833,42.276944,Framingham,MA,8
8,01730,-71.276667,42.491389,Bedford,MA,9
9,01752,-71.540556,42.346111,Marlborough,MA,10


In [9]:
# Weekly orders
orders = pd.read_csv('data/orders.csv')
# Change zipcodes to strings and pad with leading zeros
orders['FROMZIP'] = orders['FROMZIP'].apply(lambda x: str(int(x)).zfill(5))
orders['TOZIP'] = orders['TOZIP'].apply(lambda x: str(int(x)).zfill(5))

orders.head(10)

,ORDERID,FROMZIP,TOZIP,CUBE,DayOfWeek,ST required?
0,0,01887,01887,(Depot),(Depot),(Depot)
1,1,01887,01801,333,Wed,no
2,2,01887,01821,556,Fri,no
3,3,01887,01843,903,Fri,no
4,4,01887,01845,111,Tue,no
5,5,01887,01867,206,Thu,no
6,6,01887,01876,2699,Wed,no
7,7,01887,01887,491,Tue,no
8,8,01887,01910,100,Tue,no
9,9,01887,02139,182,Tue,no


In [10]:
# Distances
distances = pd.read_csv('data/distances.csv', header=1, index_col=1)
distances.drop('Zip', axis=1, inplace=True)  # 'Zip' col is redundant
print(f'\nMatrix Dimensions:{distances.shape}\n')
distances.head(10)


Matrix Dimensions:(123, 123)



,1,2,3,4,5,6,7,8,9,10,...,114,115,116,117,118,119,120,121,122,123
id,,,,,,,,,,,,,,,,,,,,,
1,0,19,68,82,70,74,69,84,104,83,...,116,112,116,126,110,113,123,123,121,122
2,19,0,81,68,55,60,55,70,90,69,...,97,94,97,108,91,94,104,105,103,103
3,68,81,0,17,45,41,25,43,34,36,...,161,157,161,171,156,157,167,168,166,166
4,82,68,17,0,32,18,11,20,35,13,...,148,144,148,158,144,145,155,155,153,154
5,70,55,45,32,0,26,20,38,58,35,...,138,133,133,143,133,135,139,140,143,143
6,74,60,41,18,26,0,15,12,33,10,...,140,136,140,151,136,137,147,147,145,146
7,69,55,25,11,20,15,0,25,50,19,...,136,132,136,147,132,133,143,143,141,142
8,84,70,43,20,38,12,25,0,22,6,...,151,147,151,161,147,148,158,158,156,157
9,104,90,34,35,58,33,50,22,0,24,...,170,166,170,180,166,167,177,177,175,176


<br> The distribution center in Wilmington should be the center of the locations. This is important to keep in mind when developing the solution to the routing problem since the DC will be the starting and ending node for all the vehicles. The ID for the DC in the dataframe above is 20. This can only be determined by reading the problem statement and cross-referencing the zip code from the `orders` dataframe (which gives 01887 as the "depot" or DC zip) with the id's in `store_locs`.

In [11]:
orders.loc[0]

ORDERID               0
FROMZIP           01887
TOZIP             01887
CUBE            (Depot)
DayOfWeek       (Depot)
ST required?    (Depot)
Name: 0, dtype: object

In [12]:
store_locs.loc[store_locs['ZIP'] == '01887']

,ZIP,X,Y,CITY,STATE,ZIPID
19,01887,-71.175556,42.545556,Wilmington,MA,20


<br> The locations of the DC and stores can be used as map coordinates for plotting. The following `dict` holds the coordinates with the `ZIPID` as the key.

In [13]:
# The X, Y coordinates need to be flipped for plotting
store_loc_coords = store_locs.loc[:, ['ZIPID', 'Y', 'X']].set_index('ZIPID').T.to_dict('list')

<br> A distribution map can be created to show the locations of the DC and NHG stores.

In [14]:
# location and icon of DC (depot)
dc_loc = tuple(store_loc_coords.get(20))
dc_icon = AwesomeIcon(name='home', marker_color='red')

# initialize map centered at DC
distribution_map = Map(center=dc_loc, zoom=10)

# define the DC and stores markers
dc_marker = Marker(location=dc_loc, title='Depot-DC', icon=dc_icon, draggable=False)
store_map_locs = [tuple(store_loc_coords.get(ID)) for ID in store_loc_coords.keys() if ID != 20]
store_marker_locs = [Marker(location=crd, draggable=False) for crd in store_map_locs]
store_markers = MarkerCluster(markers=store_marker_locs)

# add the markers to the map
distribution_map.add_layer(dc_marker)
distribution_map.add_layer(store_markers)

display(distribution_map)

Map(center=[42.5455555555556, -71.1755555555556], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

---

<br> <h3><u>Creating the Daily Orders</u></h3>

A truck leaves the DC to make multiple stops and returns to the DC after distributing all the goods. There are no other pickups or deliveries along the way. For the purposes of this project, each day will be treated separately so the routes will be optimized over each day instead of over the whole week. This makes setting up the problem easier to set up for the optimization solver.

All daily orders originate from the DC. The following steps create the daily order book, dropping the origin zip (which is the same for all orders - the DC) and replacing the `TOZIP` column with the zip ID.

In [15]:
daily_orders = orders.drop(labels=0).copy(deep=True)  # drop 1st row depot --> depot
daily_orders.drop(columns='ST required?', inplace=True)  # drop last column, not used
daily_orders.CUBE = pd.to_numeric(daily_orders.CUBE)  # make sure volumes are numeric
# Use the zip indices from `store_locs`
daily_orders = pd.merge(left=daily_orders, right=store_locs.loc[:, ['ZIP', 'ZIPID']], 
                        how='inner', left_on='TOZIP', right_on='ZIP')
# Drop all zip codes
daily_orders.drop(columns=['FROMZIP', 'TOZIP', 'ZIP'], inplace=True)

In [16]:
print(f'\nTotal weekly orders:{len(daily_orders)}\n')
daily_orders.head(12)


Total weekly orders:261



,ORDERID,CUBE,DayOfWeek,ZIPID
0,1,333,Wed,12
1,37,128,Wed,12
2,126,195,Fri,12
3,2,556,Fri,13
4,21,1396,Thu,13
5,78,146,Tue,13
6,108,261,Tue,13
7,251,146,Mon,13
8,3,903,Fri,14
9,4,111,Tue,15


<br> Check the volumes and number of the daily orders:

In [17]:
daily_total_vols = daily_orders['CUBE'].groupby(daily_orders.DayOfWeek, sort=False).sum()
daily_total_vols

DayOfWeek
Wed    15192
Fri    13468
Thu    15009
Tue    11537
Mon    10223
Name: CUBE, dtype: int64

In [18]:
daily_total_dels = daily_orders['CUBE'].groupby(daily_orders.DayOfWeek, sort=False).count()
daily_total_dels

DayOfWeek
Wed    50
Fri    47
Thu    63
Tue    58
Mon    43
Name: CUBE, dtype: int64

<br> The next step is to break out the orders by day and aggregate orders for each location. The process is:
- For each day, find the total demand (i.e. required deliveries) per location.
- Create a `dict` object to store the location demand for every weekday

In [19]:
def LocDemandByDay(df=daily_orders):
    '''
    Creates a dictionary of total demand per location for each day
    ---
    returns: dict of form { 'day':{ ZIPID:total_demand } }
    '''
    days = list(daily_orders.DayOfWeek.unique())
    
    daily_demand = defaultdict(dict)
    
    for day in days:
        # filter df by day, aggregate by locations
        demand = daily_orders.loc[daily_orders.DayOfWeek == day]\
                  .groupby('ZIPID').sum().drop(columns='ORDERID')
        # convert df to dict and add to 
        daily_demand[day] = demand.to_dict().get('CUBE')
        
    return daily_demand

In [20]:
demands = LocDemandByDay()
demands

defaultdict(dict,
            {'Wed': {1: 151,
              2: 140,
              7: 159,
              8: 137,
              10: 164,
              12: 461,
              16: 159,
              18: 2800,
              26: 328,
              27: 120,
              28: 280,
              29: 420,
              35: 132,
              39: 239,
              42: 157,
              44: 176,
              45: 2077,
              54: 151,
              55: 366,
              56: 241,
              57: 2202,
              58: 212,
              63: 246,
              73: 280,
              74: 140,
              80: 286,
              84: 94,
              85: 210,
              88: 149,
              90: 246,
              92: 236,
              95: 304,
              105: 130,
              106: 283,
              107: 396,
              116: 376,
              120: 235,
              122: 197,
              123: 112},
             'Fri': {3: 444,
              4: 390,
              5: 336,

<br> <h3><u>Create Daily Time Matrices</u></h3>

This type of problem is know as a <i>capacitated vehicle routing problem with time windows</i> or <b>CVRPTW</b> problem. Because of the time windows and the time constraints imposed in the case, the matrix of distances has to be converted to a matrix of travel times between locations. The steps are:
- Create an overall time matrix (in minutes) from the given distance matrix.
- Extract a time matrix for each day from the overall time matrix.

The average vehicle speed is is assumed to be <b>40 mph</b> as given in the case study.

In [21]:
# Vehicle speed in miles per minute
veh_speed = np.round(40/60, 3)

In [22]:
travel_times = distances.applymap(lambda x: int(np.round(x/veh_speed, 0)))
travel_times.columns = travel_times.columns.astype(int, copy=False)
travel_times

,1,2,3,4,5,6,7,8,9,10,...,114,115,116,117,118,119,120,121,122,123
id,,,,,,,,,,,,,,,,,,,,,
1,0,28,102,123,105,111,103,126,156,124,...,174,168,174,189,165,169,184,184,181,183
2,28,0,121,102,82,90,82,105,135,103,...,145,141,145,162,136,141,156,157,154,154
3,102,121,0,25,67,61,37,64,51,54,...,241,235,241,256,234,235,250,252,249,249
4,123,102,25,0,48,27,16,30,52,19,...,222,216,222,237,216,217,232,232,229,231
5,105,82,67,48,0,39,30,57,87,52,...,207,199,199,214,199,202,208,210,214,214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,169,141,235,217,202,205,199,222,250,220,...,9,10,13,28,13,0,24,25,24,25
120,184,156,250,232,208,220,214,237,265,235,...,15,16,15,4,34,24,0,1,10,10
121,184,157,252,232,210,220,214,237,265,235,...,24,16,15,4,36,25,1,0,12,12


<br> Double-check that travel times make sense. Take the maximum travel time and check the distance on the map.

In [23]:
max_travel = tuple(np.where(travel_times == travel_times.max().max())[0])
store_locs.loc[(max_travel[0]+1, max_travel[1]+1), :]  # note df is indexed from 1 not 0

,ZIP,X,Y,CITY,STATE,ZIPID
71,05401,-73.220556,44.485000,Burlington,VT,72
117,06877,-73.498056,41.281389,Ridgefield,CT,118


The largest distance is between the Burlington, VT and Ridgefield, CT stores which seems logical as shown by the map. Note that the distance matrix is based on road miles, not the straight distance.

In [24]:
# initialize another map centered at DC
max_distance_map = Map(center=dc_loc, zoom=6)

# define the DC and stores markers
store1 = Marker(location=tuple(store_loc_coords.get(72)), draggable=False)
store2 = Marker(location=tuple(store_loc_coords.get(118)), draggable=False)

# add the markers to the map
max_distance_map.add_layer(store1)
max_distance_map.add_layer(store2)

display(max_distance_map)

Map(center=[42.5455555555556, -71.1755555555556], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

In [25]:
# max_distance_map.save('MaxDistMap1.html', title='Distance Map 1')

<br> Next, extract the daily time matrices using the `demands` dict that was just created. Creating the daily matrices can get tricky because:
- There are different indexes for the same location. The ZIPID's are labeled from 1 to 123 while the Numpy array will be zero-indexed and truncated for each array. The solver uses yet another indexing scheme. It is important to make sure everything is consistent.
- The depot (ZIPID 20) is not included for each day in the demand schedule <i>except</i> Tuesday, so it has be added for each day.

In [26]:
def makeTimeMtrxForDay(day, df=travel_times, schedule=demands):
    '''
    Create a time matrix for a given day based on the overall travel times
    ---
    day: day for which to create the matrix
    df: the df with the overall travel times to extract
    schedule: the schedule of deliveries (demands) for the day 
    '''
    from itertools import product
    
    locations = list(schedule.get(day).keys())  # get the delivery locations
    #  - Make sure to include the depot if it is not in the locations
    #  - It is better to have the depot at the beginning so it has index 0
    #    in the new matrix. This makes things easier later.
    if 20 not in locations:
        locations.insert(0, 20)
    if locations[0] != 20:
        locations.remove(20)
        locations.insert(0, 20)
         
    # Initialize a matrix for the values
    num_locs = len(locations)  # no.of locations to service
    time_matrix = np.zeros((num_locs, num_locs), dtype=int)
    
    # List of indices to use for inserting travel times
    indices = list(product(locations, locations))
    # Indices to map to in new array
    new_indices = list(product(range(num_locs), range(num_locs)))
    
    # Insert travel times into matrix
    for (i,j), (u,v) in zip(new_indices, indices):
        time_matrix[i, j] = travel_times.loc[u, v]
    
    return time_matrix

In [27]:
def makeTimeMatrices(df=travel_times, schedule=demands):
    '''
    Create time matrices for all days in the schedule
    '''
    time_mtrx_by_day = dict()
    
    for day in list(schedule.keys()):
        time_mtrx_by_day[day] = makeTimeMtrxForDay(day)
        
    return time_mtrx_by_day

<br> Create the daily time matrices and store in a dictionary:

In [28]:
time_matrices = makeTimeMatrices()
time_matrices

{'Wed': array([[  0, 169, 148, ..., 280, 279, 279],
        [169,   0,  28, ..., 184, 181, 183],
        [148,  28,   0, ..., 156, 154, 154],
        ...,
        [280, 184, 156, ...,   0,  10,  10],
        [279, 181, 154, ...,  10,   0,   3],
        [279, 183, 154, ...,  10,   3,   0]]),
 'Fri': array([[  0,  69,  58, ..., 264, 280, 282],
        [ 69,   0,  25, ..., 234, 250, 252],
        [ 58,  25,   0, ..., 216, 232, 232],
        ...,
        [264, 234, 216, ...,   0,  34,  36],
        [280, 250, 232, ...,  34,   0,   1],
        [282, 252, 232, ...,  36,   1,   0]]),
 'Thu': array([[  0,  46,  64, ..., 270, 264, 279],
        [ 46,   0,   9, ..., 226, 220, 235],
        [ 64,   9,   0, ..., 225, 217, 232],
        ...,
        [270, 226, 225, ...,   0,  24,  22],
        [264, 220, 217, ...,  24,   0,  22],
        [279, 235, 232, ...,  22,  22,   0]]),
 'Tue': array([[  0,  21,  12, ..., 270, 282, 279],
        [ 21,   0,   9, ..., 255, 265, 264],
        [ 12,   9,   0, ...

---

<br> <h3><u>Save Files</u></h3>

Save the prepared objects for use in the solution.

In [29]:
import pickle

gp_data = [time_matrices, 
           demands, 
           travel_times, 
           daily_orders, 
           daily_total_dels, 
           daily_total_vols]

with open('pickle/gp_data.pickle', 'wb') as f:
    pickle.dump(gp_data, f)